In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# download historical data for required stocks
ticker = 'AAPL'
AAPL = yf.download(ticker, dt.date.today() - dt.timedelta(365 * 5), dt.date.today())

In [ ]:
AAPL.head()

In [ ]:
AAPL.tail()

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.test import SMA


In [ ]:
class SmaCross(SignalStrategy, TrailingStrategy):
    n1 = 50
    n2 = 200
    
    def init(self):
        super().init()
        
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1, 0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0) # Upwards/long only
        
        entry_size = signal * 0.95
        
        self.set_signal(entry_size = entry_size)
        
        self.set_atr_periods(20)
        self.set_trailing_sl(2) # ATR stop loss

In [ ]:
from backtesting import Backtest

bt = Backtest(AAPL, SmaCross, commission=0.002, trade_on_close=True)
bt.run()
bt.plot()

In [ ]:
stats = bt.run()
stats

In [ ]:
# Parameter optimization
stats = bt.optimize(n1=range(5, 50, 5),
                        n2=range(10, 200, 5),
                        maximize='Sharpe Ratio',
                        constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
# get the optimization parameters of n1, n2
stats._strategy

In [ ]:
class SmaCross2(SignalStrategy, TrailingStrategy):
    n1 = 5
    n2 = 175
    
    def init(self):
        super().init()
        
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1, 0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0) # Upwards/long only
        
        entry_size = signal * 0.95
        
        self.set_signal(entry_size = entry_size)
        
        self.set_atr_periods(20)
        self.set_trailing_sl(2) # ATR stop loss

In [ ]:
bt = Backtest(AAPL, SmaCross2, commission=0.002, trade_on_close=True)
bt.run()
bt.plot()

In [ ]:
stats = bt.run()
stats